In [1]:
import os
import pickle
import random
import numpy as np 
import pandas as pd 
from tqdm import tqdm

import torch
from torch import nn


In [2]:
data_path = '../../data/'

train_df = pd.read_csv(data_path+'train.csv')

In [3]:
train_df

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,context,label,text,essay_text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,,1,"Lead Hi, i'm Isaac, i'm going to be writing ab...","Hi, i'm Isaac, i'm going to be writing about h..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,,1,"Position On my perspective, I think that the f...","Hi, i'm Isaac, i'm going to be writing about h..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"[SEP]Position On my perspective, I think that ...",1,Claim I think that the face is a natural landf...,"Hi, i'm Isaac, i'm going to be writing about h..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,[SEP]Claim I think that the face is a natural ...,1,"Evidence If life was on Mars, we would know by...","Hi, i'm Isaac, i'm going to be writing about h..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,[SEP]Claim I think that the face is a natural ...,1,Counterclaim People thought that the face was ...,"Hi, i'm Isaac, i'm going to be writing about h..."
...,...,...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,[SEP]Position Advice is something that can imp...,1,Claim For many people they don't like only ask...,Some people may ask multiple people for advice...
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,[SEP]Position Advice is something that can imp...,1,Claim also people have different views and opi...,Some people may ask multiple people for advice...
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,,1,Position Advice is something that can impact a...,Some people may ask multiple people for advice...
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,[SEP]Claim Advice is something that can impact...,0,Evidence someone can use everything that many ...,Some people may ask multiple people for advice...


In [4]:
# for essay_id, essay in tqdm(train_df.groupby("essay_id")):
#     try:
#         essay_position = essay.loc[essay['discourse_type'] == 'Position', 'discourse_text'].values[0]
#     except:
#         try:
#             essay_position = essay.loc[essay['discourse_type'] == 'Lead', 'discourse_text'].values[0]
#         except:
#             essay_position = ' '
#     current_claim = ' '
#     current_evidence = []
#     full_essay = ''
#     full_claims = []
#     for idx, row in essay.iterrows():
#         discourse_type = essay.loc[idx, 'discourse_type']
#         discourse_id = essay.loc[idx, 'discourse_id']
#         text = essay.loc[idx, 'discourse_text']
#         full_essay += text
        
#         if discourse_type == 'Lead':
#             train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] = ' '
#             current_claim = text
            
#         elif discourse_type == 'Position':
#             train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] = ' '
#             current_claim = text
#         elif discourse_type == 'Claim':
#             if len(essay_position) > 1:
#                 train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] = 'Position ' + essay_position
#             current_claim = text
#             current_evidence = []
#             full_claims.append(text)
            
#         elif discourse_type == 'Counterclaim':
#             train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] = 'Claim ' + current_claim
#             for evidence in current_evidence:
#                 train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] += '[SEP]Evidence ' + evidence
#             current_claim = text
#             current_evidence = []
            
#         elif discourse_type == 'Rebuttal':
#             train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] = 'Claim ' + current_claim
#             for evidence in current_evidence:
#                 train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] += '[SEP]Evidence ' + evidence
#             current_claim = text
#             current_evidence = []
        
#         elif discourse_type == 'Evidence':
#             if len(current_claim) > 1:
#                 train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] = 'Claim ' + current_claim
#             current_evidence.append(text)
        
#         elif discourse_type == 'Concluding Statement':
#             if essay_position != ' ':
#                 train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] = 'Position ' + essay_position
#             for claim in full_claims:
#                 train_df.loc[train_df['discourse_id'] == discourse_id, 'context'] += '[SEP]Claim ' + claim

In [5]:
from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

def add_separators(context):
    if len(context) > 1:
        context = '[SEP]' + context
    return context

def get_essay(essay_id, is_train=True):
    parent_path = INPUT_DIR + 'train' if is_train else INPUT_DIR + 'test'
    essay_path = os.path.join(parent_path, f"{essay_id}.txt")
    essay_text = open(essay_path, 'r').read()
    return essay_text

import re

def remove_whitespace(text):
    new_text = text
    while new_text[0] == ' ':
        new_text = new_text[1:]
    while new_text[-1] == ' ':
        new_text = new_text[:-1]
    return new_text


In [6]:
INPUT_DIR = "../../data/"
train_df['essay_text']  = train_df['essay_id'].apply(lambda x: get_essay(x, is_train=True))

In [7]:
train_df['label'] = ''
LABEL_MAPPING = {"Ineffective": 0, "Adequate": 1, "Effective": 2}
train_df['label'] = train_df['discourse_effectiveness'].apply(lambda x: LABEL_MAPPING[x])
train_df['discourse_text'] = train_df['discourse_text'].apply(lambda x : resolve_encodings_and_normalize(x))
train_df['essay_text'] = train_df['essay_text'].apply(lambda x : resolve_encodings_and_normalize(x))
train_df['discourse_text'] = train_df['discourse_text'].apply(lambda x : remove_whitespace(x))
train_df['essay_text'] = train_df['essay_text'].apply(lambda x : remove_whitespace(x))
train_df['text'] = train_df['discourse_type'] + '[SEP]' + train_df['discourse_text'] + '[SEP]' + train_df['essay_text']


In [21]:
from ipywidgets import interact

@interact
def f(idx = (0, 4120, 1)):
    print('FULL TEXT: ' + train_df.loc[idx, 'text'])
    
    

interactive(children=(IntSlider(value=2060, description='idx', max=4120), Output()), _dom_classes=('widget-int…

In [22]:
from sklearn.model_selection import StratifiedGroupKFold
gkf = StratifiedGroupKFold(n_splits=5)
train_df['fold'] = np.nan
for i,(train_index,test_index) in enumerate(gkf.split(train_df, train_df.discourse_effectiveness, train_df.essay_id)):
    train_df.loc[test_index,'fold'] = i
print(train_df.groupby(['fold'])['essay_id'].count())
print(train_df.groupby(['fold'])['essay_id'].nunique())
print(train_df.groupby('fold')['discourse_effectiveness'].value_counts())
# train_df.to_csv('../../data/train_folds.csv',index=False)

fold
0.0    7351
1.0    7360
2.0    7350
3.0    7353
4.0    7351
Name: essay_id, dtype: int64
fold
0.0    838
1.0    841
2.0    831
3.0    840
4.0    841
Name: essay_id, dtype: int64
fold  discourse_effectiveness
0.0   Adequate                   4195
      Effective                  1865
      Ineffective                1291
1.0   Adequate                   4198
      Effective                  1866
      Ineffective                1296
2.0   Adequate                   4195
      Effective                  1863
      Ineffective                1292
3.0   Adequate                   4195
      Effective                  1866
      Ineffective                1292
4.0   Adequate                   4194
      Effective                  1866
      Ineffective                1291
Name: discourse_effectiveness, dtype: int64


In [10]:
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,context,label,text,essay_text,fold
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,,1,"Lead[SEP]Hi, i'm Isaac, i'm going to be writin...","Hi, i'm Isaac, i'm going to be writing about h...",2.0
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,,1,"Position[SEP]On my perspective, I think that t...","Hi, i'm Isaac, i'm going to be writing about h...",2.0
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"[SEP]Position On my perspective, I think that ...",1,Claim[SEP]I think that the face is a natural l...,"Hi, i'm Isaac, i'm going to be writing about h...",2.0
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,[SEP]Claim I think that the face is a natural ...,1,"Evidence[SEP]If life was on Mars, we would kno...","Hi, i'm Isaac, i'm going to be writing about h...",2.0
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,[SEP]Claim I think that the face is a natural ...,1,Counterclaim[SEP]People thought that the face ...,"Hi, i'm Isaac, i'm going to be writing about h...",2.0


In [11]:
import glob, pandas as pd, numpy as np, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm

sws = stopwords.words("english") + ["n't",  "'s", "'ve"]

In [12]:
fls = glob.glob("../../data/train/*.txt")
docs = []
for fl in tqdm(fls):
    with open(fl) as f:
        txt = f.read()
        word_tokens = word_tokenize(txt)
        txt = " ".join([w for w in word_tokens if not w.lower() in sws])
    docs.append(txt)

100%|████████████████████████████████████████████████████████████████| 4191/4191 [00:08<00:00, 498.08it/s]


In [13]:
from bertopic import BERTopic
topic_model = BERTopic(n_gram_range=(1, 3), top_n_words=10, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/131 [00:00<?, ?it/s]

2022-07-29 14:16:53,171 - BERTopic - Transformed documents to Embeddings
2022-07-29 14:17:08,956 - BERTopic - Reduced dimensionality
2022-07-29 14:17:09,049 - BERTopic - Clustered reduced embeddings


In [14]:
tm_meta = topic_model.get_topic_info()
tm_meta.to_csv("topic_model_metadata.csv", index=False)
display(tm_meta)

pred_topics = pd.DataFrame()
dids = list(map(lambda fl: fl.split("/")[-1].split(".")[0], fls))
pred_topics["id"] = dids
pred_topics["Topic"] = topics
pred_topics['prob'] = probs
pred_topics = pred_topics.drop(columns={'prob'})
pred_topics = pred_topics.rename(columns={'id': 'essay_id'})
display(pred_topics)
pred_topics = pred_topics.merge(tm_meta, on='Topic', how='left')
pred_topics

,Topic,Count,Name
0,-1,75,-1_school_activity_activities_extracurricular
1,0,872,0_electoral_vote_college_electoral college
2,1,410,1_emotions_technology_computer_facial
3,2,398,2_venus_planet_earth_author
4,3,325,3_face_mars_aliens_landform
5,4,302,4_students_online_school_classes
6,5,292,5_community_service_community service_help
7,6,238,6_summer_students_project_projects
8,7,209,7_advice_people_multiple_opinions
9,8,205,8_cars_driverless_car_driverless cars


,essay_id,Topic
0,F6F4D85648D6,0
1,20162733792B,1
2,802C65A883AC,3
3,BFC4CADADD49,10
4,8892A2EB9E0C,3
...,...,...
4186,F7D1D884AAD4,13
4187,32C41C1CBA06,9
4188,0B81C3067FF2,2
4189,5C2A497012B9,4


,essay_id,Topic,Count,Name
0,F6F4D85648D6,0,872,0_electoral_vote_college_electoral college
1,20162733792B,1,410,1_emotions_technology_computer_facial
2,802C65A883AC,3,325,3_face_mars_aliens_landform
3,BFC4CADADD49,10,180,10_cars_car_usage_smog
4,8892A2EB9E0C,3,325,3_face_mars_aliens_landform
...,...,...,...,...
4186,F7D1D884AAD4,13,76,13_seagoing_program_animals_cowboys
4187,32C41C1CBA06,9,181,9_phones_cell_cell phones_school
4188,0B81C3067FF2,2,398,2_venus_planet_earth_author
4189,5C2A497012B9,4,302,4_students_online_school_classes


In [17]:
# train_df = pd.read_csv('../../data/train_folds.csv')
train_df = train_df.merge(pred_topics, on='essay_id', how='left')
train_df

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,context,label,text,essay_text,fold,Topic,Count,Name
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,,1,"Lead[SEP]Hi, i'm Isaac, i'm going to be writin...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,3_face_mars_aliens_landform
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,,1,"Position[SEP]On my perspective, I think that t...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,3_face_mars_aliens_landform
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"[SEP]Position On my perspective, I think that ...",1,Claim[SEP]I think that the face is a natural l...,"Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,3_face_mars_aliens_landform
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,[SEP]Claim I think that the face is a natural ...,1,"Evidence[SEP]If life was on Mars, we would kno...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,3_face_mars_aliens_landform
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,[SEP]Claim I think that the face is a natural ...,1,Counterclaim[SEP]People thought that the face ...,"Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,3_face_mars_aliens_landform
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,[SEP]Position Advice is something that can imp...,1,Claim[SEP]For many people they don't like only...,Some people may ask multiple people for advice...,1.0,7,209,7_advice_people_multiple_opinions
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,[SEP]Position Advice is something that can imp...,1,Claim[SEP]also people have different views and...,Some people may ask multiple people for advice...,1.0,7,209,7_advice_people_multiple_opinions
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,,1,Position[SEP]Advice is something that can impa...,Some people may ask multiple people for advice...,1.0,7,209,7_advice_people_multiple_opinions
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,[SEP]Claim Advice is something that can impact...,0,Evidence[SEP]someone can use everything that m...,Some people may ask multiple people for advice...,1.0,7,209,7_advice_people_multiple_opinions


In [18]:
def clean_topic(x):
    words = x.split('_')[1:]
    return ' '.join(words)
train_df['Name'] = train_df['Name'].apply(lambda x: clean_topic(x))
train_df

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,context,label,text,essay_text,fold,Topic,Count,Name
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,,1,"Lead[SEP]Hi, i'm Isaac, i'm going to be writin...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,,1,"Position[SEP]On my perspective, I think that t...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"[SEP]Position On my perspective, I think that ...",1,Claim[SEP]I think that the face is a natural l...,"Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,[SEP]Claim I think that the face is a natural ...,1,"Evidence[SEP]If life was on Mars, we would kno...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,[SEP]Claim I think that the face is a natural ...,1,Counterclaim[SEP]People thought that the face ...,"Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,[SEP]Position Advice is something that can imp...,1,Claim[SEP]For many people they don't like only...,Some people may ask multiple people for advice...,1.0,7,209,advice people multiple opinions
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,[SEP]Position Advice is something that can imp...,1,Claim[SEP]also people have different views and...,Some people may ask multiple people for advice...,1.0,7,209,advice people multiple opinions
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,,1,Position[SEP]Advice is something that can impa...,Some people may ask multiple people for advice...,1.0,7,209,advice people multiple opinions
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,[SEP]Claim Advice is something that can impact...,0,Evidence[SEP]someone can use everything that m...,Some people may ask multiple people for advice...,1.0,7,209,advice people multiple opinions


In [19]:
train_df['text'] = train_df['discourse_type'] + '[SEP]' + train_df['discourse_text'] + '[SEP]' + train_df['Name'] + '[SEP]' + train_df['essay_text']
train_df

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,context,label,text,essay_text,fold,Topic,Count,Name
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,,1,"Lead[SEP]Hi, i'm Isaac, i'm going to be writin...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,,1,"Position[SEP]On my perspective, I think that t...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"[SEP]Position On my perspective, I think that ...",1,Claim[SEP]I think that the face is a natural l...,"Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,[SEP]Claim I think that the face is a natural ...,1,"Evidence[SEP]If life was on Mars, we would kno...","Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,[SEP]Claim I think that the face is a natural ...,1,Counterclaim[SEP]People thought that the face ...,"Hi, i'm Isaac, i'm going to be writing about h...",2.0,3,325,face mars aliens landform
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,[SEP]Position Advice is something that can imp...,1,Claim[SEP]For many people they don't like only...,Some people may ask multiple people for advice...,1.0,7,209,advice people multiple opinions
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,[SEP]Position Advice is something that can imp...,1,Claim[SEP]also people have different views and...,Some people may ask multiple people for advice...,1.0,7,209,advice people multiple opinions
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,,1,Position[SEP]Advice is something that can impa...,Some people may ask multiple people for advice...,1.0,7,209,advice people multiple opinions
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,[SEP]Claim Advice is something that can impact...,0,Evidence[SEP]someone can use everything that m...,Some people may ask multiple people for advice...,1.0,7,209,advice people multiple opinions


In [23]:
train_df.to_csv('../../data/train_folds.csv', index=False)